# 04. サーバーレス推論エンドポイント デプロイ

## 概要
1. カスタム推論コード（inference.py）の作成
2. 推論用コンテナイメージの準備
3. SageMakerモデルの登録
4. サーバーレスエンドポイントの作成
5. 動作確認

## 1. セットアップ

In [ ]:
import boto3
import json
import os
import tarfile
import shutil
from pathlib import Path
from datetime import datetime

import sagemaker
from sagemaker.serverless import ServerlessInferenceConfig

s3 = boto3.client('s3')
sm_client = boto3.client('sagemaker')
sess = sagemaker.Session()

SAGEMAKER_BUCKET = sess.default_bucket()
ROLE = sagemaker.get_execution_role()
REGION = 'us-east-1'
ACCOUNT_ID = boto3.client('sts').get_caller_identity()['Account']

# モデルS3パス（03_training.ipynbで出力された値を設定）
MODEL_S3_URI = f's3://{SAGEMAKER_BUCKET}/yolo-pose/models/XXXXXXXX-XXXXXX/model.tar.gz'  # TODO: 実際のパスに置換

# エンドポイント名
ENDPOINT_NAME = 'facteye-meter-keypoint'
MODEL_NAME = 'facteye-meter-keypoint-model'
ENDPOINT_CONFIG_NAME = 'facteye-meter-keypoint-config'

print(f'Account: {ACCOUNT_ID}')
print(f'Region: {REGION}')
print(f'Role: {ROLE}')
print(f'Model: {MODEL_S3_URI}')

## 2. カスタム推論コード (inference.py) の作成

SageMakerの推論コンテナで実行されるカスタムコード。
- `model_fn`: モデルのロード
- `input_fn`: リクエストのデシリアライズ
- `predict_fn`: 推論実行
- `output_fn`: レスポンスのシリアライズ

In [ ]:
INFERENCE_CODE = '''
import json
import io
import os
import numpy as np
import onnxruntime as ort
from PIL import Image


def model_fn(model_dir):
    """モデルをロードする"""
    onnx_path = os.path.join(model_dir, "best.onnx")
    config_path = os.path.join(model_dir, "model_config.json")
    
    # ONNXランタイムセッション
    session = ort.InferenceSession(
        onnx_path,
        providers=["CPUExecutionProvider"]
    )
    
    # モデル設定
    with open(config_path) as f:
        config = json.load(f)
    
    return {"session": session, "config": config}


def input_fn(request_body, content_type):
    """リクエストをデシリアライズする"""
    if content_type in ("image/jpeg", "image/png"):
        image = Image.open(io.BytesIO(request_body)).convert("RGB")
        return {"image": image}
    elif content_type == "application/json":
        data = json.loads(request_body)
        # Base64エンコードされた画像の場合
        import base64
        image_bytes = base64.b64decode(data["image"])
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        return {"image": image, "metadata": data.get("metadata", {})}
    else:
        raise ValueError(f"Unsupported content type: {content_type}")


def preprocess(image, input_size=640):
    """画像を前処理する（リサイズ + 正規化）"""
    orig_w, orig_h = image.size
    
    # アスペクト比を保持してリサイズ（letterbox）
    scale = min(input_size / orig_w, input_size / orig_h)
    new_w = int(orig_w * scale)
    new_h = int(orig_h * scale)
    
    image_resized = image.resize((new_w, new_h), Image.BILINEAR)
    
    # パディング（グレーで埋める）
    canvas = Image.new("RGB", (input_size, input_size), (114, 114, 114))
    pad_x = (input_size - new_w) // 2
    pad_y = (input_size - new_h) // 2
    canvas.paste(image_resized, (pad_x, pad_y))
    
    # numpy配列に変換 + 正規化
    img_array = np.array(canvas, dtype=np.float32) / 255.0
    # HWC -> CHW
    img_array = np.transpose(img_array, (2, 0, 1))
    # バッチ次元追加
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array, {
        "orig_w": orig_w,
        "orig_h": orig_h,
        "scale": scale,
        "pad_x": pad_x,
        "pad_y": pad_y
    }


def postprocess(output, preprocess_info, config, conf_threshold=0.5):
    """推論結果を後処理する
    
    YOLOv8-poseの出力形式:
    [batch, num_detections, 5 + num_keypoints*3]
    5 = cx, cy, w, h, confidence
    num_keypoints * 3 = (x, y, visibility) per keypoint
    """
    predictions = output[0]  # shape: [1, num_detections, 5+kp*3]
    if len(predictions.shape) == 3:
        predictions = predictions[0]  # remove batch dim
    
    # 転置が必要な場合（YOLOv8の出力は [features, detections] の場合がある）
    if predictions.shape[0] < predictions.shape[1]:
        predictions = predictions.T
    
    num_keypoints = config.get("num_keypoints", 4)
    keypoint_labels = config.get("keypoint_labels", 
        ["needle_tip", "needle_center", "scale_min", "scale_max"])
    
    scale = preprocess_info["scale"]
    pad_x = preprocess_info["pad_x"]
    pad_y = preprocess_info["pad_y"]
    
    results = []
    
    for det in predictions:
        # bbox: cx, cy, w, h
        cx, cy, w, h = det[0], det[1], det[2], det[3]
        confidence = det[4]
        
        if confidence < conf_threshold:
            continue
        
        # パディングとスケールを元に戻す
        cx = (cx - pad_x) / scale
        cy = (cy - pad_y) / scale
        w = w / scale
        h = h / scale
        
        # キーポイント抽出
        keypoints = {}
        for i in range(num_keypoints):
            kp_offset = 5 + i * 3
            kp_x = (det[kp_offset] - pad_x) / scale
            kp_y = (det[kp_offset + 1] - pad_y) / scale
            kp_conf = det[kp_offset + 2]
            
            keypoints[keypoint_labels[i]] = {
                "x": float(kp_x),
                "y": float(kp_y),
                "confidence": float(kp_conf)
            }
        
        results.append({
            "bbox": {
                "cx": float(cx),
                "cy": float(cy),
                "width": float(w),
                "height": float(h)
            },
            "confidence": float(confidence),
            "keypoints": keypoints
        })
    
    # confidence降順でソート
    results.sort(key=lambda x: x["confidence"], reverse=True)
    
    return results


def predict_fn(input_data, model):
    """推論を実行する"""
    session = model["session"]
    config = model["config"]
    image = input_data["image"]
    
    input_size = config.get("input_size", 640)
    
    # 前処理
    img_array, preprocess_info = preprocess(image, input_size)
    
    # 推論
    input_name = session.get_inputs()[0].name
    output = session.run(None, {input_name: img_array})
    
    # 後処理
    detections = postprocess(output, preprocess_info, config)
    
    return {
        "detections": detections,
        "image_size": {
            "width": preprocess_info["orig_w"],
            "height": preprocess_info["orig_h"]
        },
        "metadata": input_data.get("metadata", {})
    }


def output_fn(prediction, accept):
    """レスポンスをシリアライズする"""
    if accept == "application/json":
        return json.dumps(prediction), accept
    else:
        return json.dumps(prediction), "application/json"
'''

# 推論コードをファイルに保存
inference_dir = Path('/tmp/inference_code')
inference_dir.mkdir(parents=True, exist_ok=True)

(inference_dir / 'inference.py').write_text(INFERENCE_CODE.strip())
print('inference.py 作成完了')

## 3. requirements.txt の作成

In [ ]:
REQUIREMENTS = """onnxruntime==1.17.1
numpy>=1.24.0
Pillow>=10.0.0
"""

(inference_dir / 'requirements.txt').write_text(REQUIREMENTS.strip())
print('requirements.txt 作成完了')

## 4. 推論コード付きモデルアーティファクトの作成

SageMakerでは `model.tar.gz` に推論コードを含めることができる。

```
model.tar.gz/
├── best.onnx
├── model_config.json
└── code/
    ├── inference.py
    └── requirements.txt
```

In [ ]:
# 既存のmodel.tar.gzをダウンロード
model_s3_parts = MODEL_S3_URI.replace('s3://', '').split('/', 1)
model_bucket = model_s3_parts[0]
model_key = model_s3_parts[1]

local_model_tar = Path('/tmp/model_original.tar.gz')
s3.download_file(model_bucket, model_key, str(local_model_tar))
print(f'モデルダウンロード完了: {local_model_tar}')

# 展開
extract_dir = Path('/tmp/model_extracted')
if extract_dir.exists():
    shutil.rmtree(extract_dir)
extract_dir.mkdir(parents=True)

with tarfile.open(str(local_model_tar), 'r:gz') as tar:
    tar.extractall(str(extract_dir))

print(f'展開ファイル: {list(extract_dir.rglob("*"))}')

In [ ]:
# 推論コードをcodeディレクトリに配置
code_dir = extract_dir / 'code'
code_dir.mkdir(parents=True, exist_ok=True)

shutil.copy2(inference_dir / 'inference.py', code_dir / 'inference.py')
shutil.copy2(inference_dir / 'requirements.txt', code_dir / 'requirements.txt')

# 新しいmodel.tar.gzを作成
new_model_tar = Path('/tmp/model_with_code.tar.gz')
with tarfile.open(str(new_model_tar), 'w:gz') as tar:
    for item in extract_dir.rglob('*'):
        if item.is_file():
            arcname = str(item.relative_to(extract_dir))
            tar.add(str(item), arcname=arcname)

print(f'新model.tar.gz: {new_model_tar}')
print(f'サイズ: {new_model_tar.stat().st_size / 1024 / 1024:.1f} MB')

# 内容確認
with tarfile.open(str(new_model_tar), 'r:gz') as tar:
    print('\n含まれるファイル:')
    for member in tar.getmembers():
        print(f'  {member.name} ({member.size} bytes)')

In [ ]:
# S3にアップロード
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
deploy_model_key = f'yolo-pose/deploy/{timestamp}/model.tar.gz'

s3.upload_file(
    str(new_model_tar),
    SAGEMAKER_BUCKET,
    deploy_model_key
)

DEPLOY_MODEL_S3_URI = f's3://{SAGEMAKER_BUCKET}/{deploy_model_key}'
print(f'デプロイ用モデルアップロード完了: {DEPLOY_MODEL_S3_URI}')

## 5. SageMaker モデル登録

In [ ]:
from sagemaker.image_uris import retrieve

# PyTorchの推論コンテナイメージを使用（ONNX Runtime含む）
framework_version = '2.1'
py_version = 'py310'

image_uri = retrieve(
    framework='pytorch',
    region=REGION,
    version=framework_version,
    py_version=py_version,
    instance_type='ml.m5.large',  # サーバーレスでもイメージ取得に必要
    image_scope='inference'
)

print(f'コンテナイメージ: {image_uri}')

In [ ]:
# 既存のモデル・エンドポイントを削除（再デプロイ時）
for name, delete_fn, describe_fn in [
    (ENDPOINT_NAME, sm_client.delete_endpoint, sm_client.describe_endpoint),
    (ENDPOINT_CONFIG_NAME, sm_client.delete_endpoint_config, sm_client.describe_endpoint_config),
    (MODEL_NAME, sm_client.delete_model, sm_client.describe_model),
]:
    try:
        describe_fn(**{list(describe_fn.__code__.co_varnames)[1]: name})
        delete_fn(**{list(delete_fn.__code__.co_varnames)[1]: name})
        print(f'削除: {name}')
    except sm_client.exceptions.ClientError:
        pass  # 存在しない場合は無視

In [ ]:
# SageMakerモデル作成
sm_client.create_model(
    ModelName=MODEL_NAME,
    PrimaryContainer={
        'Image': image_uri,
        'ModelDataUrl': DEPLOY_MODEL_S3_URI,
        'Environment': {
            'SAGEMAKER_PROGRAM': 'inference.py',
            'SAGEMAKER_SUBMIT_DIRECTORY': DEPLOY_MODEL_S3_URI,
        }
    },
    ExecutionRoleArn=ROLE,
    Tags=[
        {'Key': 'Project', 'Value': 'facteye'},
        {'Key': 'Component', 'Value': 'meter-keypoint'},
    ]
)

print(f'モデル作成完了: {MODEL_NAME}')

## 6. サーバーレスエンドポイント設定

### サーバーレス推論の設定パラメータ
| パラメータ | 値 | 説明 |
|-----------|-----|------|
| MemorySizeInMB | 4096 | メモリサイズ（ONNXモデル + 画像処理に十分な量） |
| MaxConcurrency | 5 | 最大同時実行数 |

**コールドスタート**: 初回呼び出しは30-60秒程度のレイテンシが発生。許容する方針。

In [ ]:
# サーバーレスエンドポイント設定
MEMORY_SIZE_MB = 4096  # 4GB
MAX_CONCURRENCY = 5

sm_client.create_endpoint_config(
    EndpointConfigName=ENDPOINT_CONFIG_NAME,
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': MODEL_NAME,
            'ServerlessConfig': {
                'MemorySizeInMB': MEMORY_SIZE_MB,
                'MaxConcurrency': MAX_CONCURRENCY,
            }
        }
    ],
    Tags=[
        {'Key': 'Project', 'Value': 'facteye'},
        {'Key': 'Component', 'Value': 'meter-keypoint'},
    ]
)

print(f'エンドポイント設定作成完了: {ENDPOINT_CONFIG_NAME}')
print(f'  メモリ: {MEMORY_SIZE_MB} MB')
print(f'  最大同時実行数: {MAX_CONCURRENCY}')

In [ ]:
# エンドポイント作成
sm_client.create_endpoint(
    EndpointName=ENDPOINT_NAME,
    EndpointConfigName=ENDPOINT_CONFIG_NAME,
    Tags=[
        {'Key': 'Project', 'Value': 'facteye'},
        {'Key': 'Component', 'Value': 'meter-keypoint'},
    ]
)

print(f'エンドポイント作成開始: {ENDPOINT_NAME}')
print('InService になるまで待機します...')

In [ ]:
# エンドポイントのステータスを待機
import time

while True:
    response = sm_client.describe_endpoint(EndpointName=ENDPOINT_NAME)
    status = response['EndpointStatus']
    print(f'Status: {status}')
    
    if status == 'InService':
        print('\nエンドポイントが利用可能になりました！')
        break
    elif status == 'Failed':
        print(f'\nエンドポイント作成に失敗しました')
        print(f'理由: {response.get("FailureReason", "unknown")}')
        break
    
    time.sleep(30)

## 7. 動作確認

In [ ]:
runtime = boto3.client('sagemaker-runtime')

# テスト画像をS3から取得
TEST_IMAGE_KEY = 'images/'  # TODO: 実際のテスト画像パスに置換

# S3から画像を1枚取得してテスト
paginator = s3.get_paginator('list_objects_v2')
test_key = None
for page in paginator.paginate(Bucket='facteye-images-20251114', Prefix='images/', MaxKeys=1):
    for obj in page.get('Contents', []):
        if obj['Key'].endswith('.jpg'):
            test_key = obj['Key']
            break
    if test_key:
        break

if test_key:
    print(f'テスト画像: {test_key}')
    response = s3.get_object(Bucket='facteye-images-20251114', Key=test_key)
    image_bytes = response['Body'].read()
    print(f'画像サイズ: {len(image_bytes)} bytes')
else:
    print('テスト画像が見つかりません')

In [ ]:
# 推論実行
if test_key and image_bytes:
    start_time = time.time()
    
    response = runtime.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        ContentType='image/jpeg',
        Accept='application/json',
        Body=image_bytes
    )
    
    elapsed = time.time() - start_time
    result = json.loads(response['Body'].read().decode('utf-8'))
    
    print(f'推論時間: {elapsed:.2f}秒')
    print(f'検出数: {len(result.get("detections", []))}')
    print(f'\n結果:')
    print(json.dumps(result, indent=2))

## 8. エンドポイント情報の保存

Lambda関数から呼び出す際に必要な情報を保存する。

In [ ]:
endpoint_info = {
    'endpoint_name': ENDPOINT_NAME,
    'model_name': MODEL_NAME,
    'endpoint_config_name': ENDPOINT_CONFIG_NAME,
    'model_s3_uri': DEPLOY_MODEL_S3_URI,
    'region': REGION,
    'memory_size_mb': MEMORY_SIZE_MB,
    'max_concurrency': MAX_CONCURRENCY,
    'deployed_at': datetime.now().isoformat(),
    'container_image': image_uri,
    'invoke_example': {
        'python': f"""import boto3\nruntime = boto3.client('sagemaker-runtime')\nresponse = runtime.invoke_endpoint(\n    EndpointName='{ENDPOINT_NAME}',\n    ContentType='image/jpeg',\n    Accept='application/json',\n    Body=image_bytes\n)"""
    }
}

# ローカルに保存
info_path = Path('/tmp/endpoint_info.json')
info_path.write_text(json.dumps(endpoint_info, indent=2))

# S3にも保存
s3.put_object(
    Bucket=SAGEMAKER_BUCKET,
    Key='yolo-pose/endpoint/endpoint_info.json',
    Body=json.dumps(endpoint_info, indent=2).encode('utf-8')
)

print('エンドポイント情報:')
print(json.dumps(endpoint_info, indent=2))

## 次のステップ

**05_inference_test.ipynb** で以下を実行:
- 複数画像での推論テスト
- キーポイントから角度・値を算出するロジックの検証
- Bedrockとの精度比較

### Lambda統合時の情報
- エンドポイント名: `facteye-meter-keypoint`
- ContentType: `image/jpeg`
- Accept: `application/json`
- レスポンス形式: `{"detections": [{"keypoints": {...}, "confidence": ...}]}`